### This script is going through all the cv images and perform image transformation and save the result as .png format.

In [1]:
## importing the modules from ETDAugmentation script
from ipynb.fs.full.ETDaugmentation import *
from imgaug import augmenters as iaa
import imgaug as ia
import pandas as pd
import numpy as np
import cv2
import glob
import re
import time
from tqdm import tqdm
import tensorflow as tf

2023-08-07 16:30:08.552903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 16:30:08.885783: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [31]:
class CV_IMG_AUG:
    
    def img_transform(self, img_files):
        self.img_files = img_files
        
        img_augmentation = iaa.Sequential([
            #iaa.AdditiveGaussianNoise(scale=(10, 50)),
            #iaa.SaltAndPepper(p=0.1),
            #iaa.GaussianBlur(sigma=0.5),
            iaa.LinearContrast(alpha=1),
        ], random_order = True)
        
        for n, images in tqdm(enumerate(self.img_files[0:]), desc = 'Processing augmented images'):
            aug_image = img_augmentation(image=cv2.imread(images))
            cv2.imwrite('./Label-CurriculumVitae/521/aug{}.png'.format(n),aug_image)

In [32]:
if __name__ == "__main__":
    
    start = time.time()
    with tf.device('/gpu:0'):
        cv = CV_IMG_AUG()
        
        etd_data = pd.read_csv('/home/mchou001/ResNET/500ETD.csv')
        etd_data.set_index("class", inplace = True)
        
        label_cv = etd_data.loc["Label-CurriculumVitae"]
        img_path = label_cv['img_path']
    
        augmented_img = cv.img_transform(img_path)

Processing augmented images: 124it [00:11, 10.59it/s]


In [33]:
import os

# folder path
dir_path = './Label-CurriculumVitae/521/'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 124


In [34]:
text = label_cv['text']
df = pd.DataFrame(text, columns = ['text'])
df.to_csv('./Label-CurriculumVitae/cv_text_v4.csv', encoding = 'utf-8', index= None)

In [35]:
parser = Preprocessor()
image_path = sorted(glob.glob('/home/mchou001/Label-CurriculumVitae/521/*.png'), key = parser.numericalSort)

df = pd.DataFrame(image_path, columns = ['img_path'])
df.to_csv('./Label-CurriculumVitae/cv_v4.csv', encoding = 'utf-8', index= None)

In [36]:
df1 = pd.read_csv('./Label-CurriculumVitae/cv_v4.csv')
df2 = pd.read_csv('./Label-CurriculumVitae/cv_text_v4.csv')

In [37]:
df3 = pd.concat([df1, df2], axis = 1)
df3['class'] = np.nan
df3 = df3.fillna("Label-CurriculumVitae")
df3.to_csv('./Label-CurriculumVitae/cv_img_aug_v4.csv', index = None)